In [1]:
# Machine Learning Pipeline Execution
# Este notebook ejecuta secuencialmente todos los pasos del pipeline ML

# Importaciones necesarias
import os
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Asegurarse de que podemos importar desde la raíz del proyecto
# (ajusta esto si es necesario según la estructura de tu proyecto)
sys.path.append('..')



In [2]:
import os
print(f"Directorio de trabajo actual: {os.getcwd()}")
# Si necesitas cambiar el directorio:
#os.chdir('/ruta/a/tu/proyecto')

Directorio de trabajo actual: c:\Users\natus\Documents\Trabajo\PEDRO_PEREZ\Proyecto_Mercado_de_Valores\SP500_INDEX_Analisis\notebooks


In [8]:
import os
import shutil
import sys

# 1. Detectar la estructura del proyecto de forma dinámica
# Obtener el directorio actual
current_dir = os.getcwd()
print(f"Directorio actual: {current_dir}")

# Determinar la ruta al directorio raíz del proyecto
# Asumiendo que estamos en un notebook dentro del proyecto
if 'SP500_INDEX_Analisis' in current_dir:
    # Buscar el directorio raíz del proyecto (subiendo hasta encontrar SP500_INDEX_Analisis)
    path_parts = current_dir.split('SP500_INDEX_Analisis')
    project_root = os.path.join(path_parts[0], 'SP500_INDEX_Analisis')
else:
    # Si no estamos en la estructura esperada, usamos el directorio actual
    project_root = current_dir

print(f"Raíz del proyecto detectada: {project_root}")

# 2. Definir rutas relativas al project_root
pipelines_dir = os.path.join(project_root, "pipelines")
ml_dir = os.path.join(pipelines_dir, "ml")
config_path = os.path.join(pipelines_dir, "Data Engineering.xlsx")

# Verificar si el archivo de configuración existe
if not os.path.exists(config_path):
    print(f"ADVERTENCIA: El archivo de configuración no se encuentra en: {config_path}")
else:
    print(f"Archivo de configuración encontrado en: {config_path}")

# 3. Asegurar que existan los directorios de datos
data_root = os.path.join(project_root, "Data", "0_raw")
if not os.path.exists(data_root):
    os.makedirs(data_root, exist_ok=True)
    print(f"Creado directorio de datos: {data_root}")

# 4. Configurar path para importaciones
sys.path.append(project_root)
sys.path.append(ml_dir)

# 5. Ejecutar los procesadores de datos
try:
    from step_0_preprocess import run_economic_data_processor, ejecutar_myinvestingreportnormal, run_fred_data_processor, ejecutar_otherdataprocessor
    
    print("\nEjecutando procesadores de datos con rutas relativas:")
    print("-------------------------------------------")
    
    # Ejecutar cada procesador con la ruta correcta del archivo de configuración
    run_economic_data_processor(config_file=config_path, data_root=data_root)
    ejecutar_myinvestingreportnormal(config_file=config_path, data_root=data_root)
    run_fred_data_processor(config_file=config_path, data_root=data_root)
    ejecutar_otherdataprocessor(config_file=config_path, data_root=data_root)
    
except ImportError as e:
    print(f"Error al importar funciones de preprocesamiento: {e}")
    print("\nCreando y ejecutando un script temporal con las rutas relativas...")
    
    # Crear un script temporal con las rutas relativas
    temp_script = os.path.join(current_dir, "temp_preprocess.py")
    script_content = f"""
import os
import sys

# Configurar rutas relativas
project_root = r"{project_root}"
sys.path.append(project_root)
sys.path.append(os.path.join(project_root, "pipelines", "ml"))

from pipelines.ml.step_0_preprocess import run_economic_data_processor, ejecutar_myinvestingreportnormal, run_fred_data_processor, ejecutar_otherdataprocessor

# Ejecutar con las rutas relativas
config_path = os.path.join(project_root, "pipelines", "Data Engineering.xlsx")
data_root = os.path.join(project_root, "Data", "0_raw")

# Verificar que el archivo de configuración existe
if not os.path.exists(config_path):
    print(f"ERROR: No se encuentra el archivo de configuración en: {{config_path}}")
    sys.exit(1)
else:
    print(f"Usando archivo de configuración: {{config_path}}")

# Ejecutar los procesadores
run_economic_data_processor(config_file=config_path, data_root=data_root)
ejecutar_myinvestingreportnormal(config_file=config_path, data_root=data_root)
run_fred_data_processor(config_file=config_path, data_root=data_root)
ejecutar_otherdataprocessor(config_file=config_path, data_root=data_root)
"""
    with open(temp_script, "w") as f:
        f.write(script_content)
    
    # Ejecutar el script temporal
    print(f"Ejecutando script temporal: {temp_script}")
    %run {temp_script}

2025-05-22 00:02:48,864 - INFO - ================================================================================
2025-05-22 00:02:48,866 - INFO - INICIANDO PROCESO: EconomicDataProcessor
2025-05-22 00:02:48,866 - INFO - Archivo de configuración: c:\Users\natus\Documents\Trabajo\PEDRO_PEREZ\Proyecto_Mercado_de_Valores\SP500_INDEX_Analisis\pipelines\Data Engineering.xlsx
2025-05-22 00:02:48,867 - INFO - Directorio raíz de datos: c:\Users\natus\Documents\Trabajo\PEDRO_PEREZ\Proyecto_Mercado_de_Valores\SP500_INDEX_Analisis\Data\0_raw
2025-05-22 00:02:48,868 - INFO - Fecha y hora: 2025-05-22 00:02:48
2025-05-22 00:02:48,868 - INFO - ================================================================================
2025-05-22 00:02:48,869 - INFO - Leyendo archivo de configuración...
2025-05-22 00:02:48,923 - INFO - Se encontraron 21 configuraciones para procesar
2025-05-22 00:02:48,924 - INFO - 
Procesando: US_ISM_Manufacturing (business_confidence)
2025-05-22 00:02:48,925 - INFO - - Archivo:

Directorio actual: c:\Users\natus\Documents\Trabajo\PEDRO_PEREZ\Proyecto_Mercado_de_Valores\SP500_INDEX_Analisis\notebooks
Raíz del proyecto detectada: c:\Users\natus\Documents\Trabajo\PEDRO_PEREZ\Proyecto_Mercado_de_Valores\SP500_INDEX_Analisis
Archivo de configuración encontrado en: c:\Users\natus\Documents\Trabajo\PEDRO_PEREZ\Proyecto_Mercado_de_Valores\SP500_INDEX_Analisis\pipelines\Data Engineering.xlsx

Ejecutando procesadores de datos con rutas relativas:
-------------------------------------------


2025-05-22 00:02:49,035 - INFO - - Valores no nulos en TARGET: 136
2025-05-22 00:02:49,036 - INFO - - Periodo: 2014-01-02 a 2025-04-01
2025-05-22 00:02:49,037 - INFO - - Cobertura: 100.00%
2025-05-22 00:02:49,039 - INFO - 
Procesando: US_ISM_Services (business_confidence)
2025-05-22 00:02:49,040 - INFO - - Archivo: US_ISM_Services.xlsx
2025-05-22 00:02:49,040 - INFO - - Columna TARGET: ACTUAL
2025-05-22 00:02:49,041 - INFO - - Ruta encontrada: c:\Users\natus\Documents\Trabajo\PEDRO_PEREZ\Proyecto_Mercado_de_Valores\SP500_INDEX_Analisis\Data\0_raw\business_confidence\US_ISM_Services.xlsx
2025-05-22 00:02:49,079 - INFO - - Filas encontradas: 136
2025-05-22 00:02:49,084 - INFO - Preferencia de dayfirst para c:\Users\natus\Documents\Trabajo\PEDRO_PEREZ\Proyecto_Mercado_de_Valores\SP500_INDEX_Analisis\Data\0_raw\business_confidence\US_ISM_Services.xlsx: True
2025-05-22 00:02:49,134 - WARNING - No se encontró 'ACTUAL', se usará 'Actual'
2025-05-22 00:02:49,138 - INFO - - Valores no nulos en 

KeyboardInterrupt: 

In [50]:
pip install openpyxl

Note: you may need to restart the kernel to use updated packages.


In [21]:
# Paso 0: Preprocesamiento inicial
print("Ejecutando paso 0: Preprocesamiento inicial")
%run ../pipelines/ml/step_0_preprocess.py

2025-05-13 11:58:57,062 - INFO - ================================================================================
2025-05-13 11:58:57,063 - INFO - INICIANDO PROCESO: EconomicDataProcessor
2025-05-13 11:58:57,064 - INFO - Archivo de configuración: c:\Users\natus\Documents\Trabajo\PEDRO_PEREZ\Proyecto_Mercado_de_Valores\SP500_INDEX_Analisis\pipelines/Data Engineering.xlsx
2025-05-13 11:58:57,065 - INFO - Directorio raíz de datos: c:\Users\natus\Documents\Trabajo\PEDRO_PEREZ\Proyecto_Mercado_de_Valores\SP500_INDEX_Analisis\data/0_raw
2025-05-13 11:58:57,066 - INFO - Fecha y hora: 2025-05-13 11:58:57
2025-05-13 11:58:57,066 - INFO - ================================================================================
2025-05-13 11:58:57,067 - INFO - Leyendo archivo de configuración...
2025-05-13 11:58:57,095 - INFO - Se encontraron 21 configuraciones para procesar


Ejecutando paso 0: Preprocesamiento inicial


2025-05-13 11:58:57,096 - INFO - 
Procesando: US_ISM_Manufacturing (business_confidence)
2025-05-13 11:58:57,098 - INFO - - Archivo: US_ISM_Manufacturing.xlsx
2025-05-13 11:58:57,098 - INFO - - Columna TARGET: ACTUAL
2025-05-13 11:58:57,099 - INFO - - Ruta encontrada: c:\Users\natus\Documents\Trabajo\PEDRO_PEREZ\Proyecto_Mercado_de_Valores\SP500_INDEX_Analisis\data/0_raw\business_confidence\US_ISM_Manufacturing.xlsx
2025-05-13 11:58:57,115 - INFO - - Filas encontradas: 136
2025-05-13 11:58:57,120 - INFO - Preferencia de dayfirst para c:\Users\natus\Documents\Trabajo\PEDRO_PEREZ\Proyecto_Mercado_de_Valores\SP500_INDEX_Analisis\data/0_raw\business_confidence\US_ISM_Manufacturing.xlsx: True
2025-05-13 11:58:57,176 - WARNING - No se encontró 'ACTUAL', se usará 'Actual'
2025-05-13 11:58:57,179 - INFO - - Valores no nulos en TARGET: 136
2025-05-13 11:58:57,179 - INFO - - Periodo: 2014-01-02 a 2025-04-01
2025-05-13 11:58:57,180 - INFO - - Cobertura: 100.00%
2025-05-13 11:58:57,181 - INFO - 
P

Proceso completado exitosamente


2025-05-13 11:59:00,735 - INFO - Ejemplos de fechas convertidas para Australia_10Y_Bond: [Timestamp('2025-04-30 00:00:00'), Timestamp('2025-04-29 00:00:00'), Timestamp('2025-04-28 00:00:00'), Timestamp('2025-04-24 00:00:00'), Timestamp('2025-04-23 00:00:00')]
2025-05-13 11:59:00,738 - INFO - Formato numérico detectado para Australia_10Y_Bond: americano
2025-05-13 11:59:00,743 - INFO - Para Australia_10Y_Bond (columna Date), la fecha mínima es 2014-01-01 00:00:00 y la fecha máxima es 2025-04-30 00:00:00
2025-05-13 11:59:00,745 - INFO - - Australia_10Y_Bond: 3832 filas procesadas, periodo: 2014-01-01 a 2025-04-30
2025-05-13 11:59:00,746 - INFO - 
Procesando: Italy_10Y_Bond (bond)
2025-05-13 11:59:00,747 - INFO - - Archivo: Italy_10Y_Bond
2025-05-13 11:59:00,748 - INFO - - Columna TARGET: PRICE
2025-05-13 11:59:00,750 - INFO - - Ruta encontrada: c:\Users\natus\Documents\Trabajo\PEDRO_PEREZ\Proyecto_Mercado_de_Valores\SP500_INDEX_Analisis\data/0_raw\bond\Italy_10Y_Bond.csv
2025-05-13 11:59

...

Proceso completado exitosamente


2025-05-13 11:59:32,717 - INFO - Guardando estadísticas de los indicadores...
2025-05-13 11:59:33,150 - INFO - Archivo guardado exitosamente: c:\Users\natus\Documents\Trabajo\PEDRO_PEREZ\Proyecto_Mercado_de_Valores\SP500_INDEX_Analisis\data/0_raw/datos_economicos_other_procesados.xlsx
2025-05-13 11:59:33,151 - INFO - 
2025-05-13 11:59:33,151 - INFO - RESUMEN DE EJECUCIÓN
2025-05-13 11:59:33,152 - INFO - ==================================================
2025-05-13 11:59:33,153 - INFO - Proceso: OtherDataProcessor
2025-05-13 11:59:33,153 - INFO - Tiempo de ejecución: 0.77 segundos
2025-05-13 11:59:33,154 - INFO - Archivos procesados: 4
2025-05-13 11:59:33,154 - INFO - Archivos con error: 2
2025-05-13 11:59:33,155 - INFO - Archivos procesados correctamente: 2
2025-05-13 11:59:33,155 - INFO - Periodo de datos: 2001-07-31 a 2025-04-30
2025-05-13 11:59:33,156 - INFO - Datos combinados: 8675 filas, 5 columnas
2025-05-13 11:59:33,156 - INFO - Archivo de salida: c:\Users\natus\Documents\Trabaj

Proceso completado exitosamente


2025-05-13 11:59:35,081 - INFO - Archivo Excel cargado exitosamente desde C:\Users\natus\Documents\Trabajo\PEDRO_PEREZ\Proyecto_Mercado_de_Valores\SP500_INDEX_Analisis\Data\1_preprocess\MERGEDEXCELS.xlsx
2025-05-13 11:59:35,081 - INFO - Iniciando la categorización de columnas...
2025-05-13 11:59:35,083 - INFO - Proceso completado exitosamente.


columnas encontradas para Sin categoría:
fecha

columnas encontradas para bond:
PRICE_Australia_10Y_Bond_bond
PRICE_Italy_10Y_Bond_bond
PRICE_Japan_10Y_Bond_bond
PRICE_UK_10Y_Bond_bond
PRICE_Germany_10Y_Bond_bond
PRICE_Canada_10Y_Bond_bond
PRICE_China_10Y_Bond_bond
DGS10_US_10Y_Treasury_bond
DGS2_US_2Y_Treasury_bond
AAA_Corporate_Bond_AAA_Spread_bond
BAA10YM_Corporate_Bond_BBB_Spread_bond
BAMLH0A0HYM2_High_Yield_Bond_Spread_bond

columnas encontradas para commodities:
PRICE_CrudeOil_WTI_commodities
PRICE_Gold_Spot_commodities
PRICE_Silver_Spot_commodities
PRICE_Copper_Futures_commodities
PRICE_Platinum_Spot_commodities

columnas encontradas para exchange_rate:
PRICE_EUR_USD_Spot_exchange_rate
PRICE_GBP_USD_Spot_exchange_rate
PRICE_JPY_USD_Spot_exchange_rate
PRICE_CNY_USD_Spot_exchange_rate
PRICE_AUD_USD_Spot_exchange_rate
PRICE_CAD_USD_Spot_exchange_rate
PRICE_MXN_USD_Spot_exchange_rate
PRICE_EUR_GBP_Cross_exchange_rate

columnas encontradas para index_pricing:
PRICE_S&P500_Index_index

In [25]:
# Paso 1: Fusión de archivos Excel
print("Ejecutando paso 1: Fusión de archivos Excel")
%run ../pipelines/ml/step_1_merge_excels.py

2025-05-13 12:02:57,992 - INFO - === INICIANDO PROCESO DE COMBINACIÓN DE ARCHIVOS EXCEL ===
2025-05-13 12:02:57,997 - INFO - Se encontraron 4 archivos Excel en la carpeta.
2025-05-13 12:02:57,998 - INFO - Cargando archivo: datos_economicos_normales_procesados.xlsx


Ejecutando paso 1: Fusión de archivos Excel


2025-05-13 12:02:58,625 - INFO - Archivo datos_economicos_normales_procesados.xlsx cargado en 0.63s con 4151 filas.
2025-05-13 12:02:58,626 - INFO - Archivo datos_economicos_normales_procesados.xlsx cargado correctamente con 4151 filas
--- Logging error ---
Traceback (most recent call last):
  File "C:\Users\natus\anaconda3\Lib\logging\__init__.py", line 1163, in emit
    stream.write(msg + self.terminator)
  File "C:\Users\natus\anaconda3\Lib\encodings\cp1252.py", line 19, in encode
    return codecs.charmap_encode(input,self.errors,encoding_table)[0]
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
UnicodeEncodeError: 'charmap' codec can't encode character '\u2192' in position 78: character maps to <undefined>
Call stack:
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "c:\Users\natus\Documents\Trabajo\PEDRO_PEREZ\Proyecto_Mercado_de_Valores\SP500_INDEX_Analisis\venv_MValores\Lib\site-packages\ipykernel

In [7]:
# Paso 2: Generación de categorías
print("Ejecutando paso 2: Generación de categorías")
%run ../pipelines/ml/step_2_generate_categories.py



Ejecutando paso 2: Generación de categorías


2025-05-12 13:40:41,906 - INFO - Archivo cargado correctamente. Dimensiones: (4104, 85)
2025-05-12 13:40:41,909 - WARNING - Las siguientes columnas no se encontraron en el archivo: ['Denmark_Car_Resistrations', 'US_Car_Registrations', 'SouthAfrica_Car_Registrations', 'United_Kingdom_Car_Registrations', 'Spain_Car_Registrations', 'Singapore_NonOil_Exports', 'Japan_M2_MoneySupply', 'China_M2_MoneySupply', 'US_Industrial_Production', 'UK_Retail_Sales']
2025-05-12 13:40:41,909 - INFO - No se encontraron columnas para renombrar.
2025-05-12 13:40:41,956 - INFO - Resultados detallados de categorización guardados en: c:\Users\natus\Documents\Trabajo\PEDRO_PEREZ\Proyecto_Mercado_de_Valores\SP500_INDEX_Analisis\data\1_preprocess\DIAGNOSTICO_CATEGORIAS.xlsx
2025-05-12 13:40:41,958 - INFO - Resumen de categorización:
2025-05-12 13:40:41,958 - INFO -   - Sin categoría: 1 columnas
2025-05-12 13:40:41,959 - INFO -   - bond: 11 columnas
2025-05-12 13:40:41,960 - INFO -   - business_confidence: 7 colum

In [8]:
# Paso 3: Limpieza de columnas
print("Ejecutando paso 3: Limpieza de columnas")
%run ../pipelines/ml/step_3_clean_columns.py



Ejecutando paso 3: Limpieza de columnas


2025-05-12 13:45:02,391 - INFO - Archivo cargado correctamente. Columnas: 85
2025-05-12 13:45:02,393 - INFO - Se modificaron 5 nombres de columnas.
2025-05-12 13:45:02,394 - INFO - Renombrando: 'DNKSLRTCR03GPSAM_Denmark_Car_Registrations_MoM_car_registrations' -> 'DNKSLRTCR03GPSAM_Denmark_Car_Registrations_MoM'
2025-05-12 13:45:02,394 - INFO - Renombrando: 'USASLRTCR03GPSAM_US_Car_Registrations_MoM_car_registrations' -> 'USASLRTCR03GPSAM_US_Car_Registrations_MoM'
2025-05-12 13:45:02,394 - INFO - Renombrando: 'ZAFSLRTCR03GPSAM_SouthAfrica_Car_Registrations_MoM_car_registrations' -> 'ZAFSLRTCR03GPSAM_SouthAfrica_Car_Registrations_MoM'
2025-05-12 13:45:02,395 - INFO - Renombrando: 'GBRSLRTCR03GPSAM_United_Kingdom_Car_Registrations_MoM_car_registrations' -> 'GBRSLRTCR03GPSAM_United_Kingdom_Car_Registrations_MoM'
2025-05-12 13:45:02,396 - INFO - Renombrando: 'ESPSLRTCR03GPSAM_Spain_Car_Registrations_MoM_car_registrations' -> 'ESPSLRTCR03GPSAM_Spain_Car_Registrations_MoM'
2025-05-12 13:45:07

In [9]:
# Paso 4: Transformación de características
print("Ejecutando paso 4: Transformación de características")
%run ../pipelines/ml/step_4_transform_features.py


Ejecutando paso 4: Transformación de características
✅ Archivo Excel cargado exitosamente.
🧪 Primeras filas del archivo:
                    0                              1   \
0                fecha  PRICE_Australia_10Y_Bond_bond   
1        Sin categoría                           bond   
2  2014-01-01 00:00:00                          4.289   
3  2014-01-02 00:00:00                          4.331   
4  2014-01-03 00:00:00                          4.344   

                          2                          3   \
0  PRICE_Italy_10Y_Bond_bond  PRICE_Japan_10Y_Bond_bond   
1                       bond                       bond   
2                        NaN                        NaN   
3                        NaN                        NaN   
4                        NaN                        NaN   

                       4                            5   \
0  PRICE_UK_10Y_Bond_bond  PRICE_Germany_10Y_Bond_bond   
1                    bond                         bond   
2      

In [7]:

# Paso 5: Eliminación de relaciones
print("Ejecutando paso 5: Eliminación de relaciones")
%run ../pipelines/ml/step_5_remove_relations.py

2025-05-22 09:44:28,465 - INFO - Iniciando proceso de selección y filtrado de features para series temporales...


Ejecutando paso 5: Eliminación de relaciones


2025-05-22 09:44:49,157 - INFO - Datos cargados: 2241 filas, 1184 columnas
2025-05-22 09:44:49,158 - INFO - Target identificado: PRICE_S&P500_Index_index_pricing_Return_Target


KeyError: 'fecha'

In [11]:
!pip install catboost
!pip install feature_engine
!pip install pandas_market_calendars

In [4]:
# Paso 6: Selección FPI
print("Ejecutando paso 6: Selección FPI")
%run ../pipelines/ml/step_6_fpi_selection.py



2025-05-23 11:32:27,153 - INFO - ================================================================================
2025-05-23 11:32:27,156 - INFO - INICIANDO PROCESO DE SELECCIÓN DE FEATURES POR FPI
2025-05-23 11:32:27,160 - INFO - ================================================================================
2025-05-23 11:32:27,164 - INFO - Timestamp: 20250523_113227
2025-05-23 11:32:27,166 - INFO - Directorio de entrada: C:\Users\natus\Documents\Trabajo\PEDRO_PEREZ\Proyecto_Mercado_de_Valores\SP500_INDEX_Analisis\data\2_processed
2025-05-23 11:32:27,168 - INFO - Directorio de salida: C:\Users\natus\Documents\Trabajo\PEDRO_PEREZ\Proyecto_Mercado_de_Valores\SP500_INDEX_Analisis\data\3_trainingdata
2025-05-23 11:32:27,169 - INFO - Directorio de logs: C:\Users\natus\Documents\Trabajo\PEDRO_PEREZ\Proyecto_Mercado_de_Valores\SP500_INDEX_Analisis\logs
2025-05-23 11:32:27,171 - INFO - Archivo de log: C:\Users\natus\Documents\Trabajo\PEDRO_PEREZ\Proyecto_Mercado_de_Valores\SP500_INDEX_Analis

Ejecutando paso 6: Selección FPI


2025-05-23 11:32:31,251 - INFO - Archivo 'C:\Users\natus\Documents\Trabajo\PEDRO_PEREZ\Proyecto_Mercado_de_Valores\SP500_INDEX_Analisis\data\2_processed\ULTIMO_S&P500_final.xlsx' cargado con forma (2241, 243).
2025-05-23 11:32:31,263 - INFO - Memoria utilizada: 4.26 MB
2025-05-23 11:32:31,263 - INFO - Columnas del DataFrame original: ['date', 'MoM_PRICE_Australia_10Y_Bond_bond', 'log_diff_PRICE_Australia_10Y_Bond_bond', 'MoM_PRICE_Italy_10Y_Bond_bond', 'log_diff_PRICE_Italy_10Y_Bond_bond', 'rolling_var_PRICE_Italy_10Y_Bond_bond', 'zscore_PRICE_Italy_10Y_Bond_bond', 'MoM_PRICE_Japan_10Y_Bond_bond', 'YoY_PRICE_Japan_10Y_Bond_bond', 'log_diff_PRICE_Japan_10Y_Bond_bond', 'rolling_var_PRICE_Japan_10Y_Bond_bond', '3M_change_PRICE_Japan_10Y_Bond_bond', '6M_change_PRICE_Japan_10Y_Bond_bond', 'YTD_PRICE_Japan_10Y_Bond_bond', 'zscore_PRICE_Japan_10Y_Bond_bond', 'MoM_PRICE_UK_10Y_Bond_bond', 'log_diff_PRICE_UK_10Y_Bond_bond', '3M_change_PRICE_UK_10Y_Bond_bond', 'zscore_PRICE_UK_10Y_Bond_bond', 'M

In [13]:
!pip install lightgbm
!pip install xgboost


In [3]:
# Paso 7: Selección FPI
print("Ejecutando paso 6: Selección Filtro")
%run ../pipelines/ml/step_6.1_filtro_20days.py

Ejecutando paso 6: Selección Filtro

Rango de fechas en el archivo original:
Fecha de inicio: 2025-03-31
Fecha de fin: 2025-04-25
Total de días: 26

Rango de fechas en los 20 días seleccionados:
Fecha de inicio: 2025-03-31
Fecha de fin: 2025-04-25
Total de días: 26

Proceso completado. Archivo guardado en: C:\Users\natus\Documents\Trabajo\PEDRO_PEREZ\Proyecto_Mercado_de_Valores\SP500_INDEX_Analisis\Data\2_processed\datos_economicos_filtrados.xlsx
Columnas originales en archivo 1: 767
Columnas en archivo de referencia: 31
Columnas conservadas: 31
Número de filas en el archivo final: 20

Los datos están ordenados de la fecha más antigua (2025-03-31) a la más reciente (2025-04-25)


C:\Users\natus\Documents\Trabajo\PEDRO_PEREZ\Proyecto_Mercado_de_Valores\SP500_INDEX_Analisis\pipelines\ml\step_6.1_filtro_20days.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtrado[columna_fecha] = pd.to_datetime(df_filtrado[columna_fecha], errors='coerce')


In [9]:
# Al principio de tu notebook, ANTES DE IMPORTAR el archivo principal:

# Opción 1: Deshabilitar GPU
#USE_GPU = False
#GPU_MEMORY_LIMIT = 0

# Opción 2: Usar GPU con límite de memoria
USE_GPU = True
GPU_MEMORY_LIMIT = 4096  # 4GB

# Opción 3: Usar GPU sin límites
#USE_GPU = True
#GPU_MEMORY_LIMIT = 0

In [10]:
# Paso 7: Entrenamiento de modelos
print("Ejecutando paso 7: Entrenamiento de modelos")
%run ../pipelines/ml/step_7_0_train_models.py

Ejecutando paso 7: Entrenamiento de modelos


[I 2025-05-29 11:37:14,801] A new study created in memory with name: no-name-b7a38c84-1ec4-4a3f-84a7-96532dcc457f
[I 2025-05-29 11:37:16,625] Trial 0 finished with value: 0.040755761261344774 and parameters: {'learning_rate': 0.015895621307123103, 'depth': 7, 'iterations': 355}. Best is trial 0 with value: 0.040755761261344774.
[I 2025-05-29 11:37:22,277] Trial 1 finished with value: 0.040608554918497575 and parameters: {'learning_rate': 0.017477743988547415, 'depth': 7, 'iterations': 306}. Best is trial 1 with value: 0.040608554918497575.
[I 2025-05-29 11:37:44,488] A new study created in memory with name: no-name-ea365868-f0a8-44e2-aad3-95e66cdbd89f


Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[23]	valid_0's rmse: 0.0226928	valid_0's l2: 0.000514964
Training until validation scores don't improve for 50 rounds


[I 2025-05-29 11:37:45,240] Trial 0 finished with value: 0.039281215871238666 and parameters: {'learning_rate': 0.022786401559120074, 'max_depth': 6, 'n_estimators': 305, 'subsample': 0.9406773314761803}. Best is trial 0 with value: 0.039281215871238666.


Did not meet early stopping. Best iteration is:
[278]	valid_0's rmse: 0.0602538	valid_0's l2: 0.00363052
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[5]	valid_0's rmse: 0.034897	valid_0's l2: 0.0012178
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[23]	valid_0's rmse: 0.0226922	valid_0's l2: 0.000514935
Training until validation scores don't improve for 50 rounds


[I 2025-05-29 11:37:46,260] Trial 1 finished with value: 0.039237265160961175 and parameters: {'learning_rate': 0.023190019781340183, 'max_depth': 6, 'n_estimators': 359, 'subsample': 0.9815757572203851}. Best is trial 1 with value: 0.039237265160961175.


Did not meet early stopping. Best iteration is:
[337]	valid_0's rmse: 0.0601267	valid_0's l2: 0.00361522
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[5]	valid_0's rmse: 0.0348929	valid_0's l2: 0.00121751


c:\Users\natus\Documents\Trabajo\PEDRO_PEREZ\Proyecto_Mercado_de_Valores\SP500_INDEX_Analisis\venv_MValores\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
c:\Users\natus\Documents\Trabajo\PEDRO_PEREZ\Proyecto_Mercado_de_Valores\SP500_INDEX_Analisis\venv_MValores\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
c:\Users\natus\Documents\Trabajo\PEDRO_PEREZ\Proyecto_Mercado_de_Valores\SP500_INDEX_Analisis\venv_MValores\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
c:\Users\natus\Documents\Trabajo\PEDRO_PEREZ\Proyecto_Mercado_de_Valores\SP500_INDEX_Analisis\venv_MValores\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X 


✅ Entrenamiento completado con enfoque de tres zonas en CPU 💻
✅ Número de algoritmos procesados: 6
✅ Visualizaciones generadas en: C:\Users\natus\Documents\Trabajo\PEDRO_PEREZ\Proyecto_Mercado_de_Valores\SP500_INDEX_Analisis\data\4_results\charts
✅ Modelos guardados en: C:\Users\natus\Documents\Trabajo\PEDRO_PEREZ\Proyecto_Mercado_de_Valores\SP500_INDEX_Analisis\models
✅ Predicciones consolidadas en: C:\Users\natus\Documents\Trabajo\PEDRO_PEREZ\Proyecto_Mercado_de_Valores\SP500_INDEX_Analisis\data\4_results\all_models_predictions.csv
✅ Tabla comparativa de métricas: C:\Users\natus\Documents\Trabajo\PEDRO_PEREZ\Proyecto_Mercado_de_Valores\SP500_INDEX_Analisis\data\4_results\metrics_three_zones.csv
✅ Tiempo total de ejecución: 276.45s
=== GENERANDO FACT_PREDICTIONS CSV ===
Archivo entrada: c:\Users\natus\Documents\Trabajo\PEDRO_PEREZ\Proyecto_Mercado_de_Valores\SP500_INDEX_Analisis\Data\4_results\all_models_predictions.csv
Archivo salida: c:\Users\natus\Documents\Trabajo\PEDRO_PEREZ\Pro

In [3]:
# Paso 8: Preparación de salida
print("Ejecutando paso 8: Preparación de salida")
%run ../pipelines/ml/step_7_5_ensemble.py


2025-05-13 09:23:24,662 - INFO - Iniciando creación de ensamble greedy...
2025-05-13 09:23:24,672 - INFO - Modelo cargado: catboosttrial10
2025-05-13 09:23:24,677 - INFO - Modelo cargado: catboosttrial11
2025-05-13 09:23:24,681 - INFO - Modelo cargado: catboosttrial13
2025-05-13 09:23:24,684 - INFO - Modelo cargado: catboosttrial16
2025-05-13 09:23:24,688 - INFO - Modelo cargado: catboosttrial4
2025-05-13 09:23:24,694 - INFO - Modelo cargado: catboosttrial5
2025-05-13 09:23:24,697 - INFO - Modelo cargado: catboost_best
2025-05-13 09:23:24,707 - INFO - Modelo cargado: lightgbmtrial12
2025-05-13 09:23:24,718 - INFO - Modelo cargado: lightgbmtrial15
2025-05-13 09:23:24,726 - INFO - Modelo cargado: lightgbmtrial18
2025-05-13 09:23:24,736 - INFO - Modelo cargado: lightgbmtrial19
2025-05-13 09:23:24,742 - INFO - Modelo cargado: lightgbmtrial7
2025-05-13 09:23:24,747 - INFO - Modelo cargado: lightgbm_best
2025-05-13 09:23:24,756 - INFO - Modelo cargado: mlptrial0
2025-05-13 09:23:24,766 - INF

Ejecutando paso 8: Preparación de salida


2025-05-13 09:23:24,865 - INFO - Modelo cargado: xgboosttrial12
2025-05-13 09:23:24,889 - INFO - Modelo cargado: xgboosttrial13
2025-05-13 09:23:24,926 - INFO - Modelo cargado: xgboosttrial15
2025-05-13 09:23:24,965 - INFO - Modelo cargado: xgboosttrial19
2025-05-13 09:23:25,001 - INFO - Modelo cargado: xgboosttrial4
2025-05-13 09:23:25,037 - INFO - Modelo cargado: xgboost_best
2025-05-13 09:23:25,038 - INFO - Se cargaron 32 modelos para el ensamble.
2025-05-13 09:23:25,039 - INFO - Usando datos: ULTIMO_S&P500_final_FPI.xlsx
2025-05-13 09:23:27,219 - INFO - Datos cargados: 2826 filas, 111 columnas
2025-05-13 09:23:27,220 - INFO - Columnas en el DataFrame: ['date', '6M_change_PRICE_Australia_10Y_Bond_bond', 'zscore_PRICE_Australia_10Y_Bond_bond', 'log_diff_PRICE_Italy_10Y_Bond_bond', 'rolling_std_PRICE_Italy_10Y_Bond_bond', '3M_change_PRICE_Italy_10Y_Bond_bond', 'zscore_PRICE_Italy_10Y_Bond_bond', 'MoM_PRICE_Japan_10Y_Bond_bond', 'YoY_PRICE_Japan_10Y_Bond_bond', '3M_change_PRICE_Japan_1

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.006111 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 25653
[LightGBM] [Info] Number of data points in the train set: 2261, number of used features: 109
[LightGBM] [Info] Start training from score 2862.772342
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive g

2025-05-13 09:23:59,525 - INFO - Modelo lightgbmtrial12 entrenado en 2.53s - RMSE: 1018.2621
2025-05-13 09:23:59,526 - INFO - Entrenando modelo base lightgbmtrial15 (9/32)


...

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001479 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 25653
[LightGBM] [Info] Number of data points in the train set: 2261, number of used features: 109
[LightGBM] [Info] Start training from score 2862.772342
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive g

2025-05-13 09:24:12,146 - INFO - Modelo lightgbm_best entrenado en 1.44s - RMSE: 1014.8092
2025-05-13 09:24:12,147 - INFO - Entrenando modelo base mlptrial0 (14/32)


[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

c:\Users\natus\Documents\Trabajo\PEDRO_PEREZ\Proyecto_Mercado_de_Valores\SP500_INDEX_Analisis\venv_MValores\Lib\site-packages\numpy\_core\fromnumeric.py:86: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


ValueError: Input X contains NaN.
MLPRegressor does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

In [ ]:
# Paso 9: Preparación de salida
print("Ejecutando paso 9: Preparación de salida")
%run ../pipelines/ml/step_8_prepare_output.py


In [ ]:
# Paso 10: Preparación de salida
print("Ejecutando paso 10: Preparación de salida")
%run ../pipelines/ml/step_9_backtest.py

print("¡Pipeline ML completado!")

In [ ]:
# Paso 11: Preparación de salida
print("Ejecutando paso 11: Preparación de salida")
%run ../pipelines/ml/step_10_inference.py
